In [ ]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm # 진행상황 보여주는 진행 바

In [ ]:
### Random Seed & Parameters
def set_seed(seed = 1469):
    random.seed(seed) # 일반 seed
    np.random.seed(seed) # numpy 난수 고정
    torch.manual_seed(seed) # CPU 난수 고정
    os.environ["PYTHONHASHSEED"] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(1467)

In [ ]:
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 10
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
os.chdir("/content/drive/MyDrive/3. Grad School/LG Aimers")
train = pd.read_csv("DATA/train/train.csv")

#### 모델 정의

In [ ]:
class MultiOutputLSTM(nn.Module):
        def __init__(self, input_dim = 1, hidden_dim = 64, num_layers = 2, output_dim = 7):
            """ 7개 값 예측 (PREDICT 만큼의 날짜의 값을 예측하고자 함)"""
            super(MultiOutputLSTM, self).__init__()
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first = True)
            self.fc = nn.Linear(hidden_dim, output_dim)

        def forward(self, x):
            out, _ = self.lstm(x)
            return self.fc(out[:, -1, :]) # 마지막 시점 출력만 선택해서 fc에 넣음 -> (batch * output_dim)

#### Train

In [ ]:
def train_lstm(train_df):
    """
    영업장, 메뉴별로 LSTM 모델 훈련, 각각을 trained_models에 저장
    1. 전체 데이터를 '영업장명_메뉴명'으로 나누고 -> 각 데이터를 정규화, LSTM 학습
    """
    trained_models = {}

    # store_menu : 영업장명_메뉴명 / group : 나머지 데이터
    for store_menu, group in tqdm(train_df.groupby(["영업장명_메뉴명"]), desc = "Training LSTM"):

        # 날짜 순으로 정렬해서 데이터가 너무 적으면 -> 학습 생략
        store_train = group.sort_values("영업일자").copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        # 매출 수량을 정규화
        features = ["매출수량"]
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        train_vals = store_train[features].values

        # LOOKBACK : 과거 데이터 며칠 쓸지
        # PREDICT : 미래 데이터 며칠 예측할 건지
        # 입력과 출력 (x_train, y_train) 생성
        x_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            x_train.append(train_vals[i : i + LOOKBACK])
            y_train.append(train_vals[i + LOOKBACK : i + LOOKBACK + PREDICT, 0])

        # 텐서 변환
        x_train = torch.tensor(x_train).float().to(DEVICE)
        y_train = torch.tensor(y_train).float().to(DEVICE)

        # 모델 초기화 (영업장_메뉴별로 다른 모델)
        model = MultiOutputLSTM(input_dim = 1, output_dim = PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
        criterion = nn.MSELoss()

        # 학습 모드로 설정
        model.train()

        # EPOCHS 만큼 훈련
        for epoch in range(EPOCHS):
            # idx : 랜덤하게 섞인 index들
            idx = torch.randperm(len(x_train))
            for i in range(0, len(x_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE] # 배치 개수만큼 끊어서
                x_batch, y_batch = x_train[batch_idx], y_train[batch_idx] # 배치 데이터 할당
                output = model(x_batch) # 모델 태워서
                loss = criterion(output, y_batch) # 평가하고
                optimizer.zero_grad() # 역전파를 위한 초기화
                loss.backward() # 역전파
                optimizer.step() # 최적화

        # 모델 저장
        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]  # (28, 1)
        }

    return trained_models

In [ ]:
# 학습
trained_models = train_lstm(train)

Training LSTM:   0%|          | 0/193 [00:00<?, ?it/s]/tmp/ipython-input-2896877216.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  x_train = torch.tensor(x_train).float().to(DEVICE)
Training LSTM: 100%|██████████| 193/193 [02:56<00:00,  1.10it/s]


In [ ]:
        # 예측하고자 하는 날들
        def make_variables_for_testing(self, date, predict):
            """date : 최종 날짜 (입력 7일 중 가장 마지막)"""
            future_dates = [date + timedelta(days = i + 1) for i in range(predict)]

            future_df = pd.DataFrame({'영업일자' : future_dates})

            # 파생변수 생성
            future_df = self.make_variables(future_df)

            return future_df

In [ ]:
def predict_lstm(test_df, trained_models, test_prefix : str, cols : list):
    """
    Input : test_df - test data, trained_models - list(menu : { model}), cols - x 변수들
    Output : [영업일자, 영업장명_메뉴명, 매출수량] DataFrame
    """
    results = []

    # 매장, 메뉴별로 그룹화해서 예측
    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        # 훈련된 모델에 메뉴가 있는 경우만 진행
        if store_menu not in trained_models:
            continue

        # 모델, scaler 불러오기
        model = trained_models[store_menu]['model']
        scaler = trained_models[store_menu]['scaler']

        # LSTM 입력으로 활용할 최근 lookback 만큼의 데이터 가져오기
        mv = Make_Variables()
        store_test = mv.make_variables(store_test)
        store_test_sorted = store_test.sort_values('영업일자')

        features = cols + ["매출수량"]
        if len(store_test_sorted) < lookback:
            continue

        recent_df = store_test_sorted[features].iloc[-lookback:].copy()
        if len(recent_df) < lookback:
            continue # lookback 만큼의 데이터가 없으면 예측 안 하고 넘어가기

        ##### 요기서 변수 추가
        last_date = store_test_sorted['영업일자'].iloc[-1]
        future_df = mv.make_variables_for_testing(last_date, predict)
        future_df['매출수량'] = 0.0
        full_df = pd.concat([recent_df, future_df[features]], axis = 0)

        # 정규화
        pp = Preprocess_LSTM()
        full_df = pp.label_encoding_lstm(full_df, cols)
        full_df, _ = pp.minmax_scaling_lstm(full_df, scaler)
        # full_df['매출수량'] = scaler.transform(full_df[['매출수량']])
        x_input_vals = full_df[cols].values
        x_input = x_input_vals[:lookback]
        x_input = torch.tensor([x_input]).float().to(device)

        # 예측 수행
        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역정규화
        restored = []
        for i in range(predict):
            dummy = np.zeros((1, len(features)))
            dummy[0, features.index("매출수량")] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, features.index("매출수량")]
            restored.append(max(restored_val, 0)) # 음수 나오면 0으로 처리

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val
            })

    return pd.DataFrame(results)

In [ ]:
def predict_lstm(test_df, trained_models, test_prefix : str):
    results = []

    # 매장, 메뉴별로 그룹화해서 예측
    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        # 훈련된 모델에 메뉴가 있는 경우만 진행
        if key not in trained_models:
            continue

        # 모델, scaler 불러오기
        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        # LSTM 입력으로 활용할 최근 LOOKBACK 만큼의 데이터 가져오기
        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted['매출수량'].values[-LOOKBACK : ]
        if len(recent_vals) < LOOKBACK:
            continue # LOOKBACK 만큼의 데이터가 없으면 예측 안 하고 넘어가기

        # 정규화
        recent_vals = scaler.transform(recent_vals.reshape(-1, 1))
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        # 예측 수행
        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역정규화
        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 1))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0)) # 음수 나오면 0으로 처리

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val
            })

    return pd.DataFrame(results)

In [ ]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('DATA/test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_lstm(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)

full_pred_df = pd.concat(all_preds, ignore_index=True)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/valida

In [ ]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량']
    ))

    final_df = sample_submission.copy()

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:  # 메뉴명들
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)

    return final_df

In [ ]:
sample_submission = pd.read_csv('DATA/sample_submission.csv')
submission = convert_to_submission_format(full_pred_df, sample_submission)
submission.to_csv('baseline_submission.csv', index=False, encoding='utf-8-sig')